In [17]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson.json_util import dumps
import base64

# Import your AnimalShelter class
from ModuleFiveAnimalShelter import AnimalShelter

# Data Manipulation / Model
# Fix your username and password, and configure your AnimalShelter instance
username = "Michael"
password = "password"
shelter = AnimalShelter(username, password)

# Read data from MongoDB collection into a Pandas DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column to avoid issues in the data table
df.drop(columns=['_id'], inplace=True)

# ...

# Dashboard Layout / View
app = JupyterDash('SimpleExample')

#logo

image_filename = '/home/michaelneff_snhu/Module 6 cs340/Module 5 cs340/image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Michael Neff'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable='single',  # Allow selecting only one row at a time
        selected_rows=[0],  # Initialize with the first row selected
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
    # INSERT THE GRAPH CODE HERE
    html.Div(
        id='graph-id',
        className='col s12 m6',
        children=[
            dcc.Graph(id='your-graph-id')  # Replace 'your-graph-id' with an appropriate ID
        ]
    ),
])


html.Div([
    dcc.Dropdown(
        id='rescue-type-dropdown',
        options=[
            {'label': 'Adoption', 'value': 'Adoption'},
            {'label': 'Transfer', 'value': 'Transfer'},
            # Add more options as needed
        ],
        multi=True  # Allow multiple selections
    ),
]),



# Interaction Between Components / Controller
# Callback to highlight selected rows in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#function to update the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the map chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin TX is  dashat [30.75, -97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]




# Run the server
app.run_server(debug=True)


Hello you! Module 6 connected!
Dash app running on http://127.0.0.1:16958/
